<a href="https://colab.research.google.com/github/sinugarc/PracticaSpark/blob/main/Destino_preferido.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark
import pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=97f1792a75f5acdeb16c51682f34f9bca787a8499f56e438231892451ca7635e
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [81]:
import json
from datetime import datetime
from pyspark import SparkContext
import functools
import os

def getTpla(x):
  tpla = (x['id'],
          x['name'],
          float(x['latitude']),
          float(x['longitude']))
  return tpla

def identifyZone(x,min_lat,cte_lat,min_long,cte_long):
    zona_lat=(x[2]-min_lat)//cte_lat
    zona_long=(x[3]-min_long)//cte_long
    zona = zona_lat*5 + zona_long
    return (int(zona),x[0])

def agruparZona(sc,filename,n):
    #with SparkContext() as sc:
        sc.setLogLevel("ERROR")
        data = sc.textFile(filename)
        mov = data.map(lambda x: json.loads(x)).take(1)[0]["stations"] #quitar take y arreglar resto
            
        estaciones = list(map(getTpla,mov))
        est = map(getTpla,mov)
        
        max_long=max(estaciones,key=(lambda x: x[3]))[3]
        min_long=min(estaciones,key=(lambda x: x[3]))[3]
        max_lat=max(estaciones,key=(lambda x: x[2]))[2]
        min_lat=min(estaciones,key=(lambda x: x[2]))[2]
        
        cte_long=(max_long-min_long)/float(n)
        cte_lat=(max_lat-min_lat)/float(n)
        
        lista_zonas=list(map(lambda x : identifyZone(x,min_lat,cte_lat,min_long,cte_long),est))
        a=sc.parallelize(lista_zonas)
        a=a.groupByKey().mapValues(lambda x : list(x))

           
        print(a.collectAsMap())
        return a.collectAsMap()
    
def getTpla2(x):
  tpla = (x['idunplug_station'],
           x['idplug_station'],
           x['travel_time'])
  return tpla
  
def elegir_preferido(lista, perc, opcion):
    total=0
    for i in lista:
      total+=i[0]
    j=0
    L=[]
    if opcion==1:
      while lista[j][0]>perc*total:
          print(lista[j][1])
          L.append(lista[j][1])
          j+=1
    else:
      acum2=0
      while acum2<perc*total:
            acum2+=lista[j][0]
            L.append(lista[j][1])
            j+=1
  
    return total, L
    

def cambiar_de_id_zona(ida, dict_lista_zonas):
    for key in dict_lista_zonas:
      if ida[1] in dict_lista_zonas[key]:
        return (key,1)

def F(sc, zona_a_analizar, lista_zonas, infile1, outfile, perc, opcion):
    rdd_base = sc.textFile(infile1)
    bicis = rdd_base.map(lambda x: json.loads(x))
    movimientos = bicis.map(getTpla2)\
                  .filter(lambda x: x[2] >= 700 and x[2]<=1000 ) 
    print('1',movimientos)
    id_zona= lista_zonas[zona_a_analizar]  
    rdd= movimientos.filter(lambda x: x[0] in id_zona)\
                    .map(lambda x: cambiar_de_id_zona(x,lista_zonas))\
                    .groupByKey()\
                    .mapValues(lambda x: len(x))\
                    .map(lambda x: (x[1],x[0]))\
                    .sortByKey(False)
    print(rdd.collect())
    total, L=elegir_preferido(rdd.collect(), perc ,opcion)

    outf = open(outfile, "w")
    outf.write(f'El total de movimientos es {total} que salen de la zona {zona_a_analizar} \n')
    if opcion==1:
      outf.write(f'Las zonas preferidas con porcentaje mayor que {perc} son: \n')
    else:
      outf.write(f'Las zonas que suman al menos un porcentaje mayor que {perc} son: \n')
    for line in L:
        outf.write(str(line) + '\n')
    
    outf.close()


def main(infile1,infile2,outfile,zona,perc, opcion):
    #sc=SparkContext()
    b= agruparZona(sc,infile2,5)
    F(sc,zona, b, infile1,outfile,perc, opcion)

if __name__ == '__main__':
    main('202007_movements.json', '202007.json', 'prueba.txt', 12, 0.10, 1)
    #Pasar un diccionario con los datos
    #Escribirlo con parametros 
    #arreglar lo del sc
    #Readme
    

{6: [1, 2, 3, 10, 19, 25, 26, 27, 28, 29, 32, 35, 36, 37, 38, 39, 43, 44, 45, 46, 47, 48, 56, 57, 60, 218, 219], 12: [9, 11, 20, 30, 92, 93, 95, 97, 98, 99, 100, 108, 109, 110, 111, 112, 113, 130, 164, 165, 167, 198, 206], 10: [15, 61, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 131, 132, 207], 8: [65, 66, 67, 74, 75, 78, 79, 80, 87, 88, 189], 14: [106, 192, 193, 195, 196, 201], 2: [126, 127, 128, 136, 176, 180, 184, 185], 16: [161, 157, 156, 149, 150, 209, 210, 212], 18: [155, 148, 144, 202, 203, 204], 22: [143, 141, 140, 137, 138, 173, 216, 217], 0: [177, 220], 4: [187], 11: [4, 5, 6, 7, 8, 12, 13, 14, 16, 17, 18, 58, 59, 62, 63, 169, 163, 168, 208], 7: [21, 31, 33, 54, 55, 64, 69, 71, 72, 73, 84, 85, 86, 91, 107], 5: [34, 40, 41, 42, 49, 174, 175], 1: [50, 51, 52, 53, 129, 133, 134, 135, 178, 179, 181, 182, 183], 9: [76, 77, 190], 3: [81, 82, 83, 89, 186, 188], 13: [90, 94, 96, 101, 102, 103, 104, 105, 114, 115, 166, 170, 171, 191, 194, 199, 200], 15: [160, 197, 211], 17: [153